## Descripción del proyecto
Al operador de telecomunicaciones Interconnect le gustaría poder pronosticar su tasa de cancelación de clientes. Si se descubre que un usuario o usuaria planea irse, se le ofrecerán códigos promocionales y opciones de planes especiales. El equipo de marketing de Interconnect ha recopilado algunos de los datos personales de sus clientes, incluyendo información sobre sus planes y contratos.

Servicios de Interconnect

Interconnect proporciona principalmente dos tipos de servicios:

1. Comunicación por teléfono fijo. El teléfono se puede conectar a varias líneas de manera simultánea.
2. Internet. La red se puede configurar a través de una línea telefónica (DSL, *línea de abonado digital*) o a través de un cable de fibra óptica.

Algunos otros servicios que ofrece la empresa incluyen:

- Seguridad en Internet: software antivirus (*ProtecciónDeDispositivo*) y un bloqueador de sitios web maliciosos (*SeguridadEnLínea*).
- Una línea de soporte técnico (*SoporteTécnico*).
- Almacenamiento de archivos en la nube y backup de datos (*BackupOnline*).
- Streaming de TV (*StreamingTV*) y directorio de películas (*StreamingPelículas*)

La clientela puede elegir entre un pago mensual o firmar un contrato de 1 o 2 años. Puede utilizar varios métodos de pago y recibir una factura electrónica después de una transacción.

### Condiciones:

Característica objetivo: la columna `'EndDate'` es igual a `'No'`.

Métrica principal: AUC-ROC.

Métrica adicional: exactitud.

Criterios de evaluación:

- AUC-ROC < 0.75 — 0 SP
- 0.75 ≤ AUC-ROC < 0.81 — 4 SP
- 0.81 ≤ AUC-ROC < 0.85 — 4.5 SP
- 0.85 ≤ AUC-ROC < 0.87 — 5 SP
- 0.87 ≤ AUC-ROC < 0.88 — 5.5 SP
- AUC-ROC ≥ 0.88 — 6 SP

### Descripción de los datos 

Los datos consisten en archivos obtenidos de diferentes fuentes:

- `contract.csv` — información del contrato;
- `personal.csv` — datos personales del cliente;
- `internet.csv` — información sobre los servicios de Internet;
- `phone.csv` — información sobre los servicios telefónicos.

En cada archivo, la columna `customerID` (ID de cliente) contiene un código único asignado a cada cliente. 
- La información del contrato es válida a partir del 1 de febrero de 2020.

## Análisis exploratorio de datos (Python)
### Etapas
Como analista de datos, el trabajo consiste en analizar conjuntos de datos para extraer información valiosa que permita tomar decisiones informadas. Este proceso implica varias etapas cruciales, entre ellas:

Una descripción general de los datos

El preprocesamiento de los datos

La prueba de hipótesis

### Inicialización e importación de librerias 
    - Importar librerías de pandas para crear graficas
    - Leer el conjunto de datos utilizamos pd.read_csv

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt # Graficos
import numpy as np # Prueba de Hipotesis
from scipy.stats import chi2_contingency
import seaborn as sns # Grafico Barras

from sklearn.model_selection import train_test_split, cross_val_score # datos en entrenamiento, validación y uno de prueba
from sklearn.preprocessing import StandardScaler, LabelEncoder # estandarización de datos 

from sklearn.metrics import f1_score, roc_auc_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV

from scipy.stats import pearsonr, spearmanr # Correlación por Raking
# from sklearn.tree import DecisionTreeClassifier # Arbol de desiciones
from sklearn.metrics import roc_auc_score # Calcular el AUC-ROC
from sklearn.metrics import roc_curve # Calcular la curva ROC
from sklearn.metrics import recall_score # Calcular Recall
from sklearn.metrics import precision_score # precision_score
from sklearn.metrics import accuracy_score 
from sklearn.utils import shuffle # Submuestreo
from sklearn.preprocessing import StandardScaler, MinMaxScaler # estandarizar los datos 

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

import catboost as cb
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error, r2_score # Calcular RMSE
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, auc

### Cargar datasets
    - Identificamos el delimitador sep=";" para leer adecuadamente las tablas

In [ ]:
contract = pd.read_csv('/datasets/contract.csv') 
personal = pd.read_csv('/datasets/personal.csv') 
internet = pd.read_csv('/datasets/internet.csv') 
phone = pd.read_csv('/datasets/phone.csv') 

### Exploración inicial de datos
    - Verificar info() para ver información de cada DataFrame 
    - Utlizamos head() para mostrar información del DataFrame
    - Utlizamos describe() para ver la descripción del DataFrame 

In [ ]:
# Imprime la información general/resumida sobre el DataFrame de contract
contract.head()

In [ ]:
# Imprime la información general/resumida sobre el DataFrame de personal
personal.head()

In [ ]:
# Imprime la información general/resumida sobre el DataFrame de internet
internet.head()

In [ ]:
# Imprime la información general/resumida sobre el DataFrame de phone
phone.head()

In [ ]:
# Imprime la información general/resumida sobre el DataFrame de contract
contract.info()

In [ ]:
# Imprime la información general/resumida sobre el DataFrame de personal
personal.info()

In [ ]:
# Imprime la información general/resumida sobre el DataFrame de internet
internet.info() 

In [ ]:
# Imprime la información general/resumida sobre el DataFrame de phone
phone.info()

#### Corregir los datos

In [ ]:
### Renombramos las columnas para que el código se vea más coherente con su estilo.
contract = contract.rename(columns={
    'customerID': 'customer_id', 
    'BeginDate': 'begin_date', 
    'EndDate':'end_date', 
    'Type': 'type', 
    'PaperlessBilling':'paperless_billing',
    'PaymentMethod':'payment_method',
    'MonthlyCharges': 'monthly_charges', 
    'TotalCharges': 'total_charges',   
})

### actualizar indice
contract.reset_index(drop=True, inplace=True)

In [ ]:
contract.columns

In [ ]:
# información del contrato;
contract.head()

In [ ]:
# Renombramos las columnas para que el código se vea más coherente con su estilo.
personal = personal.rename(columns={
    'customerID': 'customer_id', 
    'SeniorCitizen': 'senior_citizen', 
    'Partner': 'partner', 'Dependents': 'dependents'
})
# actualizar indice
personal.reset_index(drop=True, inplace=True)

In [ ]:
# datos personales del cliente;
personal.head()

In [ ]:
# Renombramos las columnas para que el código se vea más coherente con su estilo.
internet = internet.rename(columns={'Unnamed: 0': 'un_named'})
internet = internet.rename(columns={'id': 'customer_id'})

# actualizar indice
internet.reset_index(drop=True, inplace=True)

In [ ]:
# información sobre los servicios de Internet;
internet.head() 

In [ ]:
# Renombramos las columnas para que el código se vea más coherente con su estilo.
phone = phone.rename(columns={'customerID': 'customer_id', 'MultipleLines': 'multiple_lines'})

# actualizar indice
phone.reset_index(drop=True, inplace=True)

In [ ]:
# información sobre los servicios telefónicos.
phone.head()

In [ ]:
contract.describe()

In [ ]:
contract.describe()

In [ ]:
personal.describe()

In [ ]:
internet.describe()

In [ ]:
phone.describe()

###  Preprocesamiento de los datos
    - Reemplaza los nombres de las columnas (ponlos en minúsculas).
    - Sin espacios los nombres de las columnas
    - Se utilizan snake_case para el uso de varias palabras. 
    - Verifica y corrige los tipos de datos (por ejemplo, asegúrate de que las columnas de ID sean números enteros).
    - Identifica y completa valores ausentes con isna() sum()
    - Identifica y elimina los valores duplicados duplicated()  
    - Verifica y corrige los tipos de datos con dtype() 
    - Verificamos que los nombres esten bien escritos

#### Contract

In [ ]:
type(contract)

In [ ]:
# Verifica si hay valores nulos
contract.isnull().sum()

In [ ]:
# Verifica valores duplicados explicitos
contract.duplicated().sum()

In [ ]:
# Verificamos duplicados implicitos en la columna 'end_date' y utilizamos sorted para organizar alfabeticamente
sorted(contract['end_date'].unique()) 

In [ ]:
# Filtrar las filas donde 'EndDate' es igual a 'Yes'
contract_filtered_yes = contract[contract['end_date'] == 'Yes'].count()

# Mostrar los resultados
contract_filtered_yes

In [ ]:
# Filtrar las filas donde 'EndDate' es igual a 'No'
contract_filtered_no = contract[contract['end_date'] == 'No'].mean()

# Mostrar los resultados
contract_filtered_no

In [ ]:
# Contar valores de fecha válidos
valid_dates = pd.to_datetime(contract['end_date'], errors='coerce').notna().mean()

# Mostrar los resultados
valid_dates

Proporción de usuarios que no han cancelado la suscripción
- 61,26% que NO a  cancelado la suscripción
- 26,53% corresponden a un tipo de dato fecha cualquiera

In [ ]:
contract.dtypes

In [ ]:
# Verificamos que la columna de tiempo sea de tipo fecha
contract['begin_date'] = pd.to_datetime(contract['begin_date'])

- Analisis columna "end_date"

1. Para convertir la columna 'end_date' a valores enteros en tu DataFrame de pandas, necesitas determinar cómo deseas que los valores se interpreten como enteros:

         A) Si, por ejemplo, EndDate contiene fechas en formato string, podrías convertirlas al número de días desde una 
         fecha específica. 
         B) Sin embargo, si EndDate contiene etiquetas como "Yes" o "No", debes convertirlas a 1 y 0 respectivamente.
         
2. Como la columba 'end_date' contiene ambos valores, para convertir una columna con datos de tipo objeto en datos binarios enteros. La función lambda convierte los valores "NO" en 0 y los valores de fecha en 1.


In [ ]:
contract['end_date'] = contract['end_date'].map(lambda x: 0 if x == 'NO' else 1).astype(int)
contract.dtypes

#### Personal

In [ ]:
# Verifica si hay valores nulos
personal.isnull().sum()

In [ ]:
# Contar duplicados explícitos
personal.duplicated().sum()

In [ ]:
# Verificamos el tipo de dato 
personal.dtypes

#### Internet

In [ ]:
# Verifica si hay valores nulos
internet.isnull().sum()

In [ ]:
# Verifica si hay NO valores nulos
internet.notna().sum()

In [ ]:
# Contar duplicados explícitos
internet.duplicated().sum()

In [ ]:
# Verificamos el tipo de dato 
internet.dtypes

In [ ]:
# Metodo to_datetime para que la columna 'sesion_date' se conviertan a un valor datetime
internet['session_date'] = pd.to_datetime(internet['session_date'], format='%Y-%m-%dT')

In [ ]:
# la columna 'id' se conviertan a un valor entero
#internet['customer_id'] = internet['customer_id'].astype('int')

In [ ]:
# Verificamos el tipo de dato 
internet.dtypes

#### Phone

In [ ]:
# Verifica si hay valores nulos
phone.isnull().sum()

In [ ]:
# Contar duplicados explícitos
phone.duplicated().sum()

In [ ]:
# Verificamos el tipo de dato 
phone.dtypes

'Conclusión,
 importamos librerias de pandas para crear graficas. Leer el conjunto de datos utilizadno pd.read_csv()
 Verificamos la información para cada DataFrame utilizando head()
 
 Los encabezados de las tablas estan de acuerdo al buen estilo 
 - Todos los caracteres en minúsculas
 - Sin espacios los nombres de las columnas
 - Se utilizan snake_case para el uso de varias palabras. 

## Estudia el comportamiento de usuario

### Porcentaje (proporción) de usuarios que no han cancelado la suscripción

### Muestra la distribución de usuarios cada mes

In [ ]:
# Creamos una nueva columna 'month'
contract["month"] = contract["begin_date"].dt.to_period("M").dt.start_time
contract.head()

In [ ]:
# Muestra cuantos clientes se estan registrando por mes en promedio
contract.groupby("month")["customer_id"].count().mean()

In [ ]:
# Muestra la distribución de contract cada mes
# Define la paleta de colores
colors = ['#4c72b0', '#55a868', '#c44e52', '#8172b2', '#ccb974']

# Agrupa y cuenta los datos
data = contract.groupby('month')['customer_id'].count()

# Crea la figura y los ejes
fig, ax = plt.subplots(figsize=(12,4))

# Traza la línea
data.plot(kind='line', color=colors[0], ax=ax)

# Personaliza la trama
plt.title('Distribución de usuarios cada mes', fontsize=14)
plt.xlabel('Mes', fontsize=12)
plt.ylabel('Número de usuarios', fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
ax.yaxis.grid(True)
ax.legend(fontsize=12, loc='upper left')

# Mostra la figura
plt.show()

In [ ]:
# Seleccionar solo los datos para la fecha 01-02-2020
contract_feb = contract[contract['begin_date'] == '2020-02-01']

# Agrupar por mes y sumar los IDs de cliente
contract_feb.groupby('month')['customer_id'].count()

Conclusión, con un promedio de 91,46% usuarios registrandose por mes.
* El año que registra el mayor nuemero de usuarios es febrero 2014 con 366 usuarios
* Comparamos que la cantidad de clientes ah disminuido en febrero 2020 con solo 11 usuarios registrados

### Muestra cuantos clientes se estan registrando por mes

In [ ]:
# Creamos una nueva columna 'month'
internet["month"] = internet["session_date"].dt.to_period("M").dt.start_time

# Verificamos la nueva columna 'month'
internet.head()

In [ ]:
# Calcula el volumen del tráfico de Internet usado por cada usuario al mes. Guarda el resultado.
monthly_mb_per_user = (
    internet
    .groupby(["user_id", "month"])["customer_id"]
    .sum()
    .reset_index()
)

# Muestra la distribución de internet cada mes
internet.groupby("month")["user_id"].count().plot(kind="line", figsize=(12,4))

Conclusión, con un promedio de 91,46% usuarios registrandose por mes.
* Diciembre es el mes que mas datos de internet consumen los usuarios. 


### Prueba de hipotesis
- Hipótesis: Los clientes que utilizan servicios adicionales como ProtecciónDeDispositivo y StreamingTV tienen una tasa de cancelación menor en comparación con aquellos que no utilizan estos servicios.

Procedimiento:

Formulación de hipótesis:

Hipótesis nula ( H0): No hay diferencia en la tasa de cancelación entre los clientes que utilizan servicios adicionales y los que no.

Hipótesis alternativa ( HA): Los clientes que utilizan servicios adicionales tienen una tasa de cancelación menor.

Recolección de datos: Obtén los datos de cancelación de clientes y el uso de servicios adicionales.

Análisis estadístico: Realiza un análisis de proporciones para comparar las tasas de cancelación entre los dos grupos (usuarios de servicios adicionales y no usuarios).

Prueba de hipótesis: Utiliza una prueba de chi-cuadrado para evaluar si la diferencia en las tasas de cancelación es significativa.

- Interpretación de resultados:

Si el valor p es menor que el nivel de significancia (por ejemplo, 0.05), rechaza la hipótesis nula y acepta la hipótesis alternativa.

De esta manera, podrás determinar si los servicios adicionales influyen en la retención de clientes.

In [ ]:
# Para realizar este analisis debemos combinar los 4 dataframes existentes con una columna en comun 
# 1. contract, personal y phone tiene en comun la columna ID
# 2. Internet columna "mont" y "session_date"
# 3. La combinación de la tabla 1 con la 2 por la columna "month"

In [ ]:
# Unimos contract y personal con un 'inner' join en la columna 'customer_id' # 6361 datos non-null quedando 632 valores fuera

# Unimos contract y personal con un 'left' join en la columna 'customer_id'
merged_df = pd.merge(contract, personal, on='customer_id', how='left')

# Unimos el resultado con phone también en la columna 'customer_id'
merged_id = pd.merge(merged_df, phone, on='customer_id', how='left')

# Rellenamos los valores nulos con 0
merged_id = merged_id.fillna(0)

# Imprimimos el resumen de información del DataFrame final
merged_id.info()

In [ ]:
# Verificamos que el dataframe merged_id contiene una columna "month" con la cual lo uniremos al dataframe internet
internet.info()

In [ ]:
# Asegúrate de que ambas columnas 'month' estén en el mismo formato (por ejemplo, tipo 'object' o 'datetime')
# merged_id['month'] = pd.to_datetime(merged_id['month'], errors='coerce')
# internet['month'] = pd.to_datetime(internet['month'], errors='coerce')

# Unimos los DataFrames por la columna 'month'
merged_df = pd.merge(merged_id, internet, on='month', how='left')

# Imprimimos el resumen de información del DataFrame final
merged_df.info()


In [ ]:
merged_df.head()

In [ ]:
# Crear una columna binaria para indicar cancelación (1 si canceló, 0 si no)
merged_df['canceled'] = np.where(merged_df['end_date'] == 'canceled', 1, 0)

# Crear una columna binaria para indicar si el cliente usa servicios adicionales (1 si usa, 0 si no)
# Supongamos que 'multiple_lines' representa el uso de servicios adicionales
merged_df['uses_additional_services'] = np.where(merged_df['multiple_lines'] == 'Yes', 1, 0)

# Calcular las tasas de cancelación para ambos grupos
group_counts = pd.crosstab(merged_df['uses_additional_services'], merged_df['canceled'])

# Realizar la prueba de chi-cuadrado
chi2, p, dof, expected = chi2_contingency(group_counts)

# Imprimir resultados
print("Tabla de contingencia:")
print(group_counts)
print("\nResultado de la prueba de chi-cuadrado:")
print(f"Chi-cuadrado: {chi2:.4f}")
print(f"Valor p: {p:.4f}")

# Interpretación
alpha = 0.05
if p < alpha:
    print("\nSe rechaza la hipótesis nula. Hay evidencia de que los clientes que usan servicios adicionales tienen una tasa de cancelación diferente.")
else:
    print("\nNo se puede rechazar la hipótesis nula. No hay suficiente evidencia para afirmar que los servicios adicionales influyen en la tasa de cancelación.")


- Conlclusión:
1. En este código, combinamos los datos necesarios de los cuatro DataFrames en uno solo mediante el uso de la función  de pandas. 
2. Luego, creamos una columna en el nuevo DataFrame que nos indica si un cliente utiliza servicios adicionales.
3. A continuación, creamos una tabla de contingencia que muestra la cantidad de clientes que cancelan y la cantidad de clientes que no cancelan, desglosados por su uso de servicios adicionales.
4. Finalmente, utilizamos la función  de scipy para realizar la prueba de chi-cuadrado, que compara las tasas de cancelación entre los clientes que utilizan servicios adicionales y los que no. La función devuelve el valor chi-cuadrado, el valor p y otros datos estadísticos relevantes. Si el valor p es menor que el nivel de significancia (0.05 en este caso), rechazamos la hipótesis nula y aceptamos la hipótesis alternativa, lo que nos indica que los clientes que utilizan servicios adicionales tienen una tasa de cancelación menor.

### Analisis de distribuición de datos

#### Identificación de los 10 principales servicios adicionales más contratados por los usuarios de Interconnect.

In [ ]:
# 1. Contamos la cantidad de veces que aparece cada tipo de servicio
top_services = merged_df['type'].value_counts().head(10)

# Mostramos los 10 servicios más contratados
print(top_services)

# Graficamos la distribución de los 10 principales servicios
plt.figure(figsize=(6, 4))
sns.barplot(x=top_services.index, y=top_services.values, palette='viridis')
plt.title('Top 10 Servicios Adicionales Más Contratados', fontsize=12)
plt.xlabel('Servicio', fontsize=12)
plt.ylabel('Cantidad de Contrataciones', fontsize=12)
plt.xticks(rotation=0, ha='center')
plt.show()

#### Conclusión del análisis de los 10 principales servicios adicionales más contratados:

- El análisis de la columna type, que representa la duración de los contratos (o los servicios adicionales ofrecidos por Interconnect), revela que la mayoría de los clientes optan por el servicio "Month-to-month", con un total de 9,991,307 contrataciones. Esto indica que un gran porcentaje de clientes prefiere contratos a corto plazo, probablemente debido a la flexibilidad que ofrecen, sin compromiso a largo plazo.

- Por otro lado, las opciones de contratos a largo plazo (como "One year" con 2,299,912 contrataciones y "Two year" con 935,865 contrataciones) tienen una menor participación en comparación con los contratos mes a mes. Esto podría sugerir que una parte importante de la base de clientes es más conservadora o tiene incertidumbre en cuanto a compromisos a largo plazo, lo cual es común en servicios que ofrecen alternativas flexibles y en constante evolución como los servicios de telecomunicaciones.

- En resumen, los clientes de Interconnect parecen optar mayoritariamente por servicios flexibles y sin compromisos a largo plazo, lo que podría ser un factor clave para la estrategia de marketing y retención de la empresa, ya que refleja una preferencia por la agilidad y la capacidad de cambiar de plan o servicio de manera más fácil.

In [ ]:
# 1. Análisis del tipo de contrato por género
# Contamos la cantidad de cada tipo de contrato por género
gender_contract_dist = merged_df.groupby(['gender', 'type']).size().unstack().fillna(0)

# Visualizamos los resultados en un gráfico de barras
gender_contract_dist.plot(kind='bar', stacked=True, figsize=(6, 4), colormap='Set2')
plt.title('Distribución de Tipo de Contrato por Género', fontsize=12)
plt.xlabel('Género', fontsize=12)
plt.ylabel('Número de Contratos', fontsize=12)
plt.xticks(rotation=0)
plt.legend(title="Tipo de Contrato", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

# 2. Análisis del tipo de contrato por edad (en base a 'senior_citizen')
# Contamos la cantidad de cada tipo de contrato por si es "Senior Citizen" o no
senior_contract_dist = merged_df.groupby(['senior_citizen', 'type']).size().unstack().fillna(0)

# Visualizamos los resultados en un gráfico de barras
senior_contract_dist.plot(kind='bar', stacked=True, figsize=(6, 4), colormap='Set3')
plt.title('Distribución de Tipo de Contrato por Senior Citizen', fontsize=12)
plt.xlabel('Senior Citizen (0: No, 1: Sí)', fontsize=12)
plt.ylabel('Número de Contratos', fontsize=12)
plt.xticks(rotation=0)
plt.legend(title="Tipo de Contrato", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

# 3. Análisis conjunto del tipo de contrato por género y edad (senior_citizen)
# Realizamos un análisis cruzado entre género y senior_citizen para ver la interacción con el tipo de contrato
combined_contract_dist = merged_df.groupby(['gender', 'senior_citizen', 'type']).size().unstack().fillna(0)

# Visualizamos en un gráfico de barras apiladas
combined_contract_dist.plot(kind='bar', stacked=True, figsize=(6, 4), colormap='Set1')
plt.title('Distribución de Tipo de Contrato por Género y Senior Citizen', fontsize=10)
plt.xlabel('Género y Senior Citizen', fontsize=10)
plt.ylabel('Número de Contratos', fontsize=10)
plt.xticks(rotation=0, ha='right')
plt.legend(title="Tipo de Contrato", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()


#### Conclusiones: 

    1. Por género:
    
   La mayoría de las personas de un género específico prefieren un tipo de contrato (por ejemplo, "Month-to-month"), podrías sugerir que ese género valora la flexibilidad más que la estabilidad de un contrato a largo plazo.
   
    2. Por edad (senior_citizen):
    
   Los adultos mayores (representados por senior_citizen = 1) tienden a preferir contratos más largos (como "One year" o "Two year"), podría indicar que este grupo prefiere una mayor estabilidad y menos cambios en su servicio.
      - Los clientes más jóvenes o no adultos mayores pueden preferir la flexibilidad de contratos "Month-to-month", lo que refleja su tendencia a cambiar de proveedor o servicio con mayor frecuencia.

    3. Análisis combinado (género + edad):
    
   Este análisis puede revelar interacciones más complejas. Por ejemplo, puede ser que los hombres jóvenes prefieran contratos "Month-to-month" más que las mujeres, o que las mujeres mayores prefieran contratos a largo plazo, lo que puede reflejar diferentes patrones de comportamiento en cuanto a la estabilidad y la flexibilidad.

#### Recomendaciones basadas en los resultados:
- Estrategias de marketing personalizadas: Si se observa que ciertos grupos demográficos (por ejemplo, mujeres mayores) prefieren contratos a largo plazo, podríamos diseñar campañas dirigidas a ofrecerles planes que destaquen la estabilidad y los beneficios a largo plazo.
- Mejora de la flexibilidad: Si los contratos "Month-to-month" son más populares entre los jóvenes, podrías explorar opciones que mejoren la flexibilidad del servicio para este segmento (por ejemplo, planes con beneficios adicionales sin compromiso a largo plazo).
- Optimización de la retención: Si observas que algunos segmentos de clientes prefieren contratos más largos, podrías implementar estrategias de retención específicas, como descuentos por compromiso, para incentivar la permanencia a largo plazo.

#### Gráficos que muestren la distribución de métodos de pago utilizados por los clientes de Interconnect

In [ ]:
# Contamos la cantidad de veces que aparece cada método de pago
payment_method_distribution = merged_df['payment_method'].value_counts()

# Mostramos la distribución de los métodos de pago
print(payment_method_distribution)

# Graficamos la distribución de métodos de pago
plt.figure(figsize=(10, 4))
sns.barplot(x=payment_method_distribution.index, y=payment_method_distribution.values, palette='Set2')
plt.title('Distribución de Métodos de Pago', fontsize=12)
plt.xlabel('Método de Pago', fontsize=12)
plt.ylabel('Cantidad de Clientes', fontsize=12)
plt.xticks(rotation=0, ha='center')
plt.show()


#### Conclusión sobre la distribución de métodos de pago utilizados por los clientes de Interconnect:
La distribución de los métodos de pago utilizados por los clientes de Interconnect muestra una clara preferencia por métodos electrónicos y automáticos.

Método de pago más popular: El "Electronic check" es el método más utilizado, con 5,126,793 pagos registrados, lo que sugiere que una gran parte de los clientes prefieren esta opción, probablemente por su comodidad y la capacidad de hacer pagos rápidos y fácilmente desde sus cuentas bancarias.

Métodos tradicionales: En segundo lugar, se encuentra el "Mailed check", con 3,672,069 pagos, lo que indica que un número considerable de clientes aún opta por métodos de pago más tradicionales, aunque su popularidad es menor en comparación con los métodos electrónicos. Esto podría reflejar a un segmento de clientes menos familiarizado o cómodo con la tecnología digital.

Pagos automáticos: Los métodos de pago automáticos, como "Bank transfer (automatic)" y "Credit card (automatic)", son también utilizados, con 2,233,745 y 2,194,477 pagos respectivamente. Estos métodos indican que algunos clientes prefieren establecer pagos recurrentes automáticos, lo que podría ayudar a evitar retrasos en los pagos y facilitar la gestión de sus suscripciones.

#### Implicaciones para la empresa:
Preferencia por la comodidad: Los clientes que usan Electronic check y métodos automáticos parecen buscar comodidad y eficiencia en el proceso de pago. Interconnect podría considerar promover aún más estos métodos en sus campañas de comunicación, destacando los beneficios de pagar de manera automática.

Segmento de clientes tradicionales: Aunque la tendencia es hacia los métodos electrónicos, un segmento significativo sigue utilizando Mailed check. Esto sugiere que Interconnect debe continuar ofreciendo métodos tradicionales de pago para estos clientes, pero podría explorar incentivos para fomentar la transición hacia métodos más modernos, lo que ayudaría a reducir costos operativos asociados con el manejo de pagos físicos.

Optimización de pagos automáticos: Dado el uso relativamente alto de pagos automáticos, podría ser beneficioso para Interconnect promover el uso de la autorización bancaria automática y tarjetas de crédito automáticas a través de descuentos o beneficios exclusivos, alentando a los clientes a adoptar métodos de pago que aseguren ingresos recurrentes de manera más estable.

En resumen, los datos indican una clara inclinación hacia la digitalización y la automatización de los pagos, lo que podría facilitar una mejor experiencia para el cliente y mayor eficiencia operativa para la empresa. Sin embargo, es importante no descuidar a los clientes que aún prefieren los métodos tradicionales y encontrar formas de guiarles hacia opciones más modernas de pago.

### Los objetivos 

Necesitamos predecir si un cliente planea irse, pronto. Debemos analizar el comportamiento de los clientes y determinar qué tarifa de prepago genera más ingresos.

Entrenar un modelo con el máximo valor F1 posible. Verifica F1 para el conjunto de prueba. 
Además, debes medir la métrica AUC-ROC y compararla con el valor F1.

Característica objetivo: la columna `'end_date'` es igual a `'No'`.
Esta se encuentra en DF llamado merged_df            

#### Preparar el modelo de predicción.

   1. Preparación de los Datos:

Filtramos los datos para la característica objetivo (end_date).

Convertimos los datos categóricos a numéricos utilizando One-Hot Encoding.

Dividimos los datos en conjuntos de entrenamiento y prueba.

   2. Entrenamiento del Modelo:

Usaremos un clasificador, como Random Forest, para entrenar el modelo.

Ajustamos el modelo para maximizar el valor F1.

   3. Evaluación del Modelo:

Calculamos la métrica F1 en el conjunto de prueba.
Medimos la AUC-ROC y comparamos ambos valores.

In [ ]:
# Continuamos vemos el DF que tenemos hasta ahora 

In [ ]:
### Codificación y escalamiento de los datos.

In [ ]:
merged_df.head()

In [ ]:
df_final = merged_df[['end_date', 'type', 'paperless_billing',
       'payment_method', 'monthly_charges', 'total_charges', 'month',
       'gender', 'partner', 'dependents', 'multiple_lines',
       'mb_used', 'session_date', 'canceled', 'uses_additional_services']].copy()

In [1]:
# Valor F1
# Crea un modelo con el máximo valor F1 posible. Para aprobar la revisión, necesitas un valor F1 de al menos 0.59. Verifica F1 para el conjunto de prueba. 

## Desarrolla un modelo con la mayor exactitud posible. 
En este proyecto, el umbral de exactitud es 0.75. Usa el dataset para comprobar la exactitud.

Instrucciones del proyecto.

Segmenta los datos fuente en un conjunto de entrenamiento, uno de validación y uno de prueba.
Investiga la calidad de diferentes modelos cambiando los hiperparámetros. Describe brevemente los hallazgos del estudio.
Comprueba la calidad del modelo usando el conjunto de prueba.


### Entrenamiento del modelo 
 Dividir los datos en conjuntos de entrenamiento y prueba (80% entrenamiento, 20% prueba)
 Separa utilizando train_test_split.

##  Conclusiones 
### El mejor modelo de entrenamiento es Catboost performance para el analisis en tiempo, velocidad y exactitud.

Evaluación de los modelos:
- RMSE_LinearRegression: 
- RMSE_Catboost performance: 
- RMSE_RandomForestRegressor: 




### Modelo CatBoostRegressor

In [ ]:
# Verificamos la versión y la documentación 
# cb.__version__

### Modelo LinearRegression

### Modelo RandomForestRegressor
